# World cities exercise - solution

The goal of this exercise is to practice a common workflow for analyzing large data sets by combining Pandas and SQLAlchemy. 

This exercise uses an edited version of the world cities dataset from maxmind:

    https://www.maxmind.com/en/free-world-cities-database

For simplicity, the original data has been cleaned by removing all rows
with missing values.


In [1]:
# Useful imports.
import pandas as pd
from sqlalchemy import create_engine, func, select, Column, Integer, MetaData, String, Table

In [2]:
# Remove any existing database created in previous runs of this notebook.
import os

if os.path.isfile('world_cities.db'):
    os.unlink('world_cities.db')

**Question 1**

Using SQL Alchemy, create a new table which contains population, country and city.
Use "sqlite:///world_cities.db" as the URI, to create an sqlite database in a local file.

The table should be called "cities" and have 3 columns: "country" of type `String`, "city" of type `String`, and "population", of type `Integer`.

In [3]:
db = create_engine("sqlite:///world_cities.db")
metadata = MetaData()

cities = Table("cities", metadata,
               Column("country", String),
               Column("city", String),
               Column("population", Integer))
metadata.create_all(db)

**Question 2**

Import the data from the file `world_cities.csv` into the newly created table.

To make things interesting, let's pretend that the CSV is so large that it does not fit into memory:

1. Use Pandas' `read_csv` function to read 10000 lines at the time from the CSV file (using the `chunksize` argument). Column 0 in the file corresponds to the index.
2. Iterate over all file chunks and
3. Store each resulting data frame to the table using the `to_sql` method. (Hint: Use `if_exists='append'` to append to the DB table, and `index=False` to prevent Pandas from trying to store the index, for which we did not create a column.)

In [4]:
reader = pd.read_csv('world_cities.csv',  chunksize=10000, index_col=0)
for table in reader:
    table.to_sql('cities', db, if_exists='append', index=False)

**Question 3**

Using SQLAlchemy only, count how many rows are stored in the cities table. (There should be 47979 of them.)

In [5]:
s = select([cities.c.city]).count()
with db.connect() as conn:
    print 'Number of rows:', conn.execute(s).scalar()

Number of rows: 47979


**Question 4**

What are the cities with a population above 5 millions people?

Using SQLAlchemy, create an appropriate SQL query. Execute the query and create a DataFrame using Pandas' `read_sql` function.

Bonus: Change your query to get a dataframe sorted by the population, ideally in descending order.

In [6]:
s = select([cities]).where(cities.c.population > 5e6).order_by(cities.c.population.desc())
df = pd.read_sql(s, db)
df

,country,city,population
0,jp,tokyo,31480498
1,cn,shanghai,14608512
2,in,bombay,12692717
3,pk,karachi,11627378
4,in,delhi,10928270
5,in,new delhi,10928270
6,ph,manila,10443877
7,ru,moscow,10381288
8,kr,seoul,10323448
9,br,sao paulo,10021437


**Question 5**

Compute the number of cities per country: create a "group by" query using SQLAlchemy, and import the result in Pandas.

Bonus: time the execution of that approach and compare with loading all data into a pandas `DataFrame` and doing the same operation there.

In [7]:
s = select([func.count(cities.c.city), cities.c.country]).group_by(cities.c.country)
pd.read_sql(s, db)

,count_1,country
0,7,ad
1,3,ae
2,57,af
3,8,ag
4,1,ai
5,66,al
6,287,am
7,14,an
8,32,ao
9,84,ar


In [8]:
# Here, we do something a bit more complicated as we order by a label, and only
# count cities > 1'000'000 inhabitants
s = (
    select([func.count(cities.c.city).label("count"), cities.c.country]).
    group_by(cities.c.country).
    where(cities.c.population > 1e6).
    order_by("count")
)
pd.read_sql(s, db)


,count,country
0,1,ae
1,1,af
2,1,am
3,1,ao
4,1,at
5,1,az
6,1,be
7,1,bf
8,1,bg
9,1,by


In [9]:
# Compare doing the computations on the DB, then importing the result in Pandas,
# with importing the whole dataset in Pandas and doing all operations in Pandas.

def sql_query():
    s = (
        select([func.count(cities.c.city).label("count"), cities.c.country]).
        group_by(cities.c.country).
        where(cities.c.population > 1e6).
        order_by("count")
    )
    return pd.read_sql(s, db)

def pandas_query():
    s = select([cities])
    df = pd.read_sql(s, db)
    return df[df.population > 1e6].groupby('country').size().order()

In [10]:
%timeit sql_query()

100 loops, best of 3: 17.4 ms per loop


In [11]:
%timeit pandas_query()

10 loops, best of 3: 149 ms per loop
